# Setting up experiment

# Scrap

os.path.abspath(os.path.join('../'))

from T5ParEvo.src.data.data import Claim, Label, ClaimPredictions,GoldDataset

In [1]:
import os
import sys
from tqdm import tqdm
import argparse
from pathlib import Path
from dataclasses import dataclass
from typing import Any, Dict, List,Tuple, Optional
import json
#
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.insert(0,module_path)
print('module_path : ', module_path)
from config import SciFactT5Config
import definitions
sys.path.append(os.path.dirname(definitions.PROJECT_VARS.ROOT_DIR))
print('definitions.PROJECT_VARS.ROOT_DIR : ', definitions.PROJECT_VARS.ROOT_DIR)
print('os.path.dirname(definitions.PROJECT_VARS.ROOT_DIR) : ',os.path.dirname(definitions.PROJECT_VARS.ROOT_DIR))
#

from T5ParEvo.src.data.data import Claim, Label, ClaimPredictions,GoldDataset
from T5ParEvo.src.linguistic.ner_abbr import NEREntity, Abbreviation
from multivers import util
from multivers.data_r import ClaimDataLoaderGenerator, get_dataloader, DataLoaderGenerator
from multivers.model_r import MultiVerSModel

from T5ParEvo.target_system.multivers.multivers_interface import PredictionParams, ModelPredictor

module_path :  /home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo
root dir :  /home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo
definitions.PROJECT_VARS.ROOT_DIR :  /home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo
os.path.dirname(definitions.PROJECT_VARS.ROOT_DIR) :  /home/qudratealahyratu/research/nlp/fact_checking/my_work
/home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo


/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/pytorch_lightning/core/decorators.py:66: LightningDeprecationWarning: The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5. Please use `trainer.predict` instead for inference. The decorator was applied to `predict`
  "The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5."


import sys
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.insert(0,module_path)
sys.path.remove('/home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo/notebooks')
print(sys.path)

from T5ParEvo.src.data.data import Claim, Label, ClaimPredictions,GoldDataset


import os
os.path.abspath(os.path.join('../'))

import sys
sys.path
sys.path.remove('/home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo/notebooks')
import notebooks
print(notebooks.__version__)

print(module_path)
print(definitions.PROJECT_VARS.ROOT_DIR)
print(os.path.dirname(definitions.PROJECT_VARS.ROOT_DIR))

import pkg_resources

version = pkg_resources.get_distribution("notebooks").version

In [2]:
# This dataset to be used only for training
cfg= SciFactT5Config()
ds_train = GoldDataset(cfg.target_dataset.loc_target_dataset_corpus,
                    cfg.target_dataset.loc_target_dataset_train)
claim_train = ds_train.get_claim(39)

In [3]:

params = PredictionParams(
    checkpoint_path= "/home/qudratealahyratu/research/nlp/fact_checking/my_work/multivers/checkpoints/scifact.ckpt",
    output_file= None,#"prediction/pred_opt_scifact.jsonl",
    batch_size=5,
    device=0,
    num_workers=4,
    no_nei=False,
    force_rationale=False,
    debug=False,
)
corpus_file = cfg.target_dataset.loc_target_dataset_corpus#cfg.target_dataset.loc_target_dataset_test#"/home/qudratealahyratu/research/nlp/fact_checking/my_work/multivers/data/scifact/corpus.jsonl"
    

gold_claims = []
claims_path = cfg.target_dataset.loc_target_dataset_test#'/home/qudratealahyratu/research/nlp/fact_checking/my_work/multivers/data/scifact/claims_test_retrived.jsonl'
with open(claims_path, 'r') as f:
    for line in f:
        data = json.loads(line)
        claim = Claim(id = data['id'], claim = data['claim'], cited_docs = data['doc_ids'], evidence = {},release = None)
        gold_claims.append(claim)

#get unique claims
unique_gold_claims = Claim.get_unique_claims(gold_claims)
# Predict for unique claims
# dataloader_generator = DataLoaderGenerator(params, unique_gold_claims, corpus_file)
# dataloader = dataloader_generator.get_dataloader_by_claims()
# predictor = ModelPredictor(params, dataloader)
# prediction_formatted = predictor.run()

In [4]:
unique_gold_claims[:10]

[Example 7: 10-20% of people with severe mental disorder receive no treatment in low and middle income countries.,
 Example 8: 25% of patients with melanoma and an objective response to PD-1 blockade will experience a progression in their melanoma.,
 Example 16: 50% of patients exposed to radiation have activated markers of myofibroblasts.,
 Example 23: 8% of burn patients are admitted for hospitalization and further treatment after appearing at hospital emergency wards or outpatient clinics.,
 Example 29: A breast cancer patient's capacity to metabolize tamoxifen has no effect on treatment outcome.,
 Example 31: A country's Vaccine Alliance (GAVI) eligibility is associated with accelerated adoption of the Hub vaccine.,
 Example 33: A deficiency of folate decreases blood levels of homocysteine.,
 Example 38: A deficiency of vitamin B6 increases blood levels of homocysteine.,
 Example 59: APOE4 expression in iPSC-derived neurons results in increased tau phosphorylation.,
 Example 65: AZ

In [6]:
from T5ParEvo.src.languistic.ner_abbr import AbbreviationModel, NERModel, NEREntity, Abbreviation

ModuleNotFoundError: No module named 'T5ParEvo.src.languistic'

In [ ]:
ner_models = [
    NERModel("en_ner_craft_md"),
    NERModel("en_ner_bc5cdr_md"),
    NERModel("en_ner_bionlp13cg_md"),
    NERModel("en_ner_jnlpba_md")
]

abbr_models = [
    AbbreviationModel("en_core_sci_scibert"),
    AbbreviationModel("en_core_sci_lg"),
    AbbreviationModel("en_ner_craft_md"),
    AbbreviationModel("en_ner_bc5cdr_md"),
    AbbreviationModel("en_ner_bionlp13cg_md"),
    AbbreviationModel("en_ner_jnlpba_md")
]


nlp_sci_brt = spacy.load("en_core_sci_scibert")

nlp_sci_brt.add_pipe("abbreviation_detector")

In [5]:
import spacy
from dataclasses import dataclass
from typing import List, Dict
from tqdm import tqdm
from scispacy.abbreviation import AbbreviationDetector
activated = spacy.prefer_gpu()

In [6]:



class AbbreviationModel:
    def __init__(self, model_name: str):
        # activated = spacy.prefer_gpu()
        self.model = spacy.load(model_name)
        self.model.add_pipe("abbreviation_detector")
        self.abbreviations = set()

    def process_claim(self, claim: Claim) -> List[Abbreviation]:
        doc = self.model(claim.claim)
        abbrs = []
        for abr in doc._.abbreviations:
            self.abbreviations.add(str(abr))
            abbr = Abbreviation(
                claim_id=claim.id,
                abrv_text=str(abr),
                abr_definition=str(abr._.long_form),
                abr_model=self.model.meta['name'],
                sf_start_char=abr.start_char,
                sf_end_char=abr.end_char,
                lf_start_char=abr._.long_form.start_char,
                lf_end_char=abr._.long_form.end_char,
            )
            if (abr.end_char - abr.start_char > 0) & (abr._.long_form.end_char - abr._.long_form.start_char > 0):
                if (abr.start_char - abr._.long_form.end_char < 4) | (abr._.long_form.start_char - abr.end_char < 4):
                    abbr.abr_whole_start = min(abr.start_char, abr._.long_form.start_char)
                    abbr.abr_whole_end = max(abr.end_char, abr._.long_form.end_char)
            abbrs.append(abbr)
        return abbrs

# models = [
#     AbbreviationModel("en_core_sci_scibert"),
#     AbbreviationModel("en_core_sci_lg"),
#     AbbreviationModel("en_ner_craft_md"),
#     AbbreviationModel("en_ner_bc5cdr_md"),
#     AbbreviationModel("en_ner_bionlp13cg_md"),
#     AbbreviationModel("en_ner_jnlpba_md")
# ]

# all_sentence_abbr = []
# for cur_claim in tqdm(unique_gold_claims):
#     for model in models:
#         all_sentence_abbr.extend(model.process_claim(cur_claim))


In [7]:
from dataclasses import dataclass
from typing import List, Dict
from tqdm import tqdm



class NERModel:
    def __init__(self, model_name: str):
        self.model = spacy.load(model_name)
        self.entities = set()

    def process_claim(self, claim: Claim) -> List[NEREntity]:
        doc = self.model(claim.claim)
        entities = []
        for ent in doc.ents:
            self.entities.add(str(ent))
            entity = NEREntity(
                claim_id=claim.id,
                ner_text=str(ent),
                ner_label=ent.label_,
                ner_model=self.model.meta['name'],
                start_char=ent.start_char,
                end_char=ent.end_char
            )
            entities.append(entity)
        return entities

# models = [
#     NERModel("en_ner_craft_md"),
#     NERModel("en_ner_bc5cdr_md"),
#     NERModel("en_ner_bionlp13cg_md"),
#     NERModel("en_ner_jnlpba_md")
# ]

# all_sentence_word_ner = []
# for cur_claim in tqdm(unique_gold_claims):
#     for model in models:
#         all_sentence_word_ner.extend(model.process_claim(cur_claim))


In [8]:
class ClaimProcessor:
    def __init__(self, ner_models: List[NERModel], abbr_models: List[AbbreviationModel]):
        self.ner_models = ner_models
        self.abbr_models = abbr_models

    def process_claim(self, claim: Claim) -> Tuple[List[NEREntity], List[Abbreviation]]:
        ner_entities = []
        abbreviations = []
        for model in self.ner_models:
            ner_entities.extend(model.process_claim(claim))
        for model in self.abbr_models:
            abbreviations.extend(model.process_claim(claim))
        return ner_entities, abbreviations


In [9]:
ner_models = [
    NERModel("en_ner_craft_md"),
    NERModel("en_ner_bc5cdr_md"),
    NERModel("en_ner_bionlp13cg_md"),
    NERModel("en_ner_jnlpba_md")
]

abbr_models = [
    # AbbreviationModel("en_core_sci_scibert"),
    # AbbreviationModel("en_core_sci_lg"),
    AbbreviationModel("en_ner_craft_md"),
    AbbreviationModel("en_ner_bc5cdr_md"),
    AbbreviationModel("en_ner_bionlp13cg_md"),
    AbbreviationModel("en_ner_jnlpba_md")
]

processor = ClaimProcessor(ner_models, abbr_models)

/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_ner_craft_md' (0.5.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_ner_bc5cdr_md' (0.5.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_ner_bionlp13cg_md' (0.5.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/qudratealahyratu/anaconda3/envs/scifact/lib/

In [10]:
all_sentence_ner = []
all_sentence_abbr = []
for cur_claim in tqdm(unique_gold_claims[:]):
    ner_entities, abbreviations = processor.process_claim(cur_claim)
    all_sentence_ner.extend(ner_entities)
    all_sentence_abbr.extend(abbreviations)

  0%|          | 0/297 [00:00<?, ?it/s]

/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/scispacy/abbreviation.py:230: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)
100%|██████████| 297/297 [00:52<00:00,  5.66it/s]


In [16]:
[print(cur_claim) for cur_claim in all_sentence_ner if cur_claim.claim_id == 188]


NEREntity(claim_id=188, ner_text='Antibodies', ner_label='GO', ner_model='ner_craft_md', start_char=27, end_char=37)
NEREntity(claim_id=188, ner_text='phospholipids', ner_label='CHEBI', ner_model='ner_craft_md', start_char=80, end_char=93)
NEREntity(claim_id=188, ner_text='phospholipids', ner_label='CHEMICAL', ner_model='ner_bc5cdr_md', start_char=80, end_char=93)
NEREntity(claim_id=188, ner_text='phospholipids', ner_label='SIMPLE_CHEMICAL', ner_model='ner_bionlp13cg_md', start_char=80, end_char=93)
NEREntity(claim_id=188, ner_text='HIV-1 Neutralizing Antibodies', ner_label='PROTEIN', ner_model='ner_jnlpba_md', start_char=8, end_char=37)


[None, None, None, None, None]

In [14]:
[print(cur_claim) for cur_claim in all_sentence_abbr if cur_claim.claim_id == 188]

Abbreviation(claim_id=188, abrv_text='bnAb', abr_definition='Broadly HIV-1 Neutralizing Antibodies', abr_model='ner_craft_md', sf_start_char=39, sf_end_char=43, lf_start_char=0, lf_end_char=37, abr_whole_start=0, abr_whole_end=43)
Abbreviation(claim_id=188, abrv_text='bnAb', abr_definition='Broadly HIV-1 Neutralizing Antibodies', abr_model='ner_bc5cdr_md', sf_start_char=39, sf_end_char=43, lf_start_char=0, lf_end_char=37, abr_whole_start=0, abr_whole_end=43)
Abbreviation(claim_id=188, abrv_text='bnAb', abr_definition='Broadly HIV-1 Neutralizing Antibodies', abr_model='ner_bionlp13cg_md', sf_start_char=39, sf_end_char=43, lf_start_char=0, lf_end_char=37, abr_whole_start=0, abr_whole_end=43)
Abbreviation(claim_id=188, abrv_text='bnAb', abr_definition='Broadly HIV-1 Neutralizing Antibodies', abr_model='ner_jnlpba_md', sf_start_char=39, sf_end_char=43, lf_start_char=0, lf_end_char=37, abr_whole_start=0, abr_whole_end=43)


[None, None, None, None]

In [13]:
[print(cur_claim) for cur_claim in all_sentence_abbr if cur_claim.claim_id == 8]

[]

In [15]:
from typing import List

def merge_overlapping_entities(entities: List[NEREntity]) -> List[NEREntity]:
    # Sort entities by start index
    entities.sort(key=lambda x: x.start_char)

    merged_entities = []
    current_entity = entities[0]

    for entity in entities[1:]:
        # If this entity overlaps with the current entity, merge them
        if entity.start_char <= current_entity.end_char:
            current_entity = NEREntity(
                claim_id=current_entity.claim_id,
                ner_text=current_entity.ner_text + entity.ner_text[current_entity.end_char - entity.start_char:],
                ner_label=current_entity.ner_label,  # Keep the label of the first entity
                ner_model=current_entity.ner_model,  # Keep the model of the first entity
                start_char=current_entity.start_char,
                end_char=max(current_entity.end_char, entity.end_char)
            )
        else:
            # This entity doesn't overlap, so add the current entity to the list and start a new one
            merged_entities.append(current_entity)
            current_entity = entity

    # Don't forget to add the last entity
    merged_entities.append(current_entity)

    return merged_entities


In [19]:
from collections import defaultdict

# Create a dictionary where each key is a claim ID and the value is a list of NEREntity objects for that claim
entities_by_claim = defaultdict(list)
for entity in all_sentence_word_ner:
    entities_by_claim[entity.claim_id].append(entity)

# Now you can merge the entities for each claim
merged_entities = {}
for claim_id, entities in entities_by_claim.items():
    merged_entities[claim_id] = merge_overlapping_entities(entities)


In [21]:
merged_entities[8]

[NEREntity(claim_id=8, ner_text='patients', ner_label='ORGANISM', ner_model='ner_bionlp13cg_md', start_char=7, end_char=15),
 NEREntity(claim_id=8, ner_text='melanoma', ner_label='DISEASE', ner_model='ner_bc5cdr_md', start_char=21, end_char=29),
 NEREntity(claim_id=8, ner_text='PD-1', ner_label='GENE_OR_GENE_PRODUCT', ner_model='ner_bionlp13cg_md', start_char=59, end_char=63),
 NEREntity(claim_id=8, ner_text='melanoma', ner_label='DISEASE', ner_model='ner_bc5cdr_md', start_char=112, end_char=120)]

In [17]:
from typing import List

def merge_overlapping_abbreviations(abbreviations: List[Abbreviation]) -> List[Abbreviation]:
    # Sort abbreviations by start index
    abbreviations.sort(key=lambda x: x.sf_start_char)

    merged_abbreviations = []
    current_abbr = abbreviations[0]

    for abbr in abbreviations[1:]:
        # If this abbreviation overlaps with the current abbreviation, merge them
        if abbr.sf_start_char <= current_abbr.sf_end_char:
            current_abbr = Abbreviation(
                claim_id=current_abbr.claim_id,
                abrv_text=current_abbr.abrv_text + abbr.abrv_text[current_abbr.sf_end_char - abbr.sf_start_char:],
                abr_definition=current_abbr.abr_definition,  # Keep the definition of the first abbreviation
                abr_model=current_abbr.abr_model,  # Keep the model of the first abbreviation
                sf_start_char=current_abbr.sf_start_char,
                sf_end_char=max(current_abbr.sf_end_char, abbr.sf_end_char),
                lf_start_char=current_abbr.lf_start_char,
                lf_end_char=max(current_abbr.lf_end_char, abbr.lf_end_char),
                abr_whole_start=current_abbr.abr_whole_start,
                abr_whole_end=max(current_abbr.abr_whole_end, abbr.abr_whole_end)
            )
        else:
            # This abbreviation doesn't overlap, so add the current abbreviation to the list and start a new one
            merged_abbreviations.append(current_abbr)
            current_abbr = abbr

    # Don't forget to add the last abbreviation
    merged_abbreviations.append(current_abbr)

    return merged_abbreviations


In [21]:
from collections import defaultdict
abbr_by_claim = defaultdict(list)
for abbr in all_sentence_abbr:
    abbr_by_claim[abbr.claim_id].append(abbr)

merged_abbreviations = {}
for claim_id, abbreviations in abbr_by_claim.items():
    merged_abbreviations[claim_id] = merge_overlapping_abbreviations(abbreviations)


In [23]:
merged_abbreviations[190]

[Abbreviation(claim_id=190, abrv_text='form', abr_definition='form (PCF', abr_model='ner_craft_md', sf_start_char=72, sf_end_char=76, lf_start_char=96, lf_end_char=105, abr_whole_start=72, abr_whole_end=105),
 Abbreviation(claim_id=190, abrv_text='BSF', abr_definition='bloodstream form', abr_model='ner_craft_md', sf_start_char=78, sf_end_char=81, lf_start_char=60, lf_end_char=76, abr_whole_start=60, abr_whole_end=81),
 Abbreviation(claim_id=190, abrv_text='form', abr_definition='form (PCF', abr_model='ner_craft_md', sf_start_char=96, sf_end_char=100, lf_start_char=96, lf_end_char=105, abr_whole_start=96, abr_whole_end=105),
 Abbreviation(claim_id=190, abrv_text='PCF', abr_definition='procyclic form', abr_model='ner_craft_md', sf_start_char=102, sf_end_char=105, lf_start_char=86, lf_end_char=100, abr_whole_start=86, abr_whole_end=105)]

# Load Existing Dataframe to Prepare class

In [7]:
import pandas as pd

In [15]:
loc_df_scispacy_sentence_word_unq_ner_abr_filtered = '../data/meta/df_scispacy_sentence_word_unq_ner_abr_cust_2.pkl'
df_scispacy_sentence_word_unq_ner_abr_filtered = pd.read_pickle(loc_df_scispacy_sentence_word_unq_ner_abr_filtered)
df_scispacy_sentence_word_unq_ner_abr_filtered

,ner_text,ner_label,ner_model,claim,start_char,end_char,org_label,list_rationales,data_source,term_in_claim
0,UK,GENE_OR_GENE_PRODUCT,en_ner_bionlp13cg_md,1 in 5 million in UK have abnormal PrP positiv...,18,20,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,UK
1,PrP,GENE_OR_GENE_PRODUCT,en_ner_bionlp13cg_md,1 in 5 million in UK have abnormal PrP positiv...,35,38,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,PrP
2,genomes,SO,en_ner_craft_md,"1,000 genomes project enables mapping of genet...",6,13,SUPPORTS,"[In conclusion, uncommon or rare genetic varia...",dev,genomes
3,genetic sequence,SO,en_ner_craft_md,"1,000 genomes project enables mapping of genet...",41,57,SUPPORTS,"[In conclusion, uncommon or rare genetic varia...",dev,genetic sequence
4,variants,SO,en_ner_craft_md,"1,000 genomes project enables mapping of genet...",139,147,SUPPORTS,"[In conclusion, uncommon or rare genetic varia...",dev,variants
...,...,...,...,...,...,...,...,...,...,...
1727,β-sheet,PROTEIN,en_ner_jnlpba_md,β-sheet opening occurs during pleurotolysin po...,0,7,SUPPORTS,[The major conformational changes in PlyB are ...,train,β-sheet
1728,pleurotolysin,PROTEIN,en_ner_jnlpba_md,β-sheet opening occurs during pleurotolysin po...,30,48,SUPPORTS,[The major conformational changes in PlyB are ...,train,pleurotolysin pore
1729,cRCT,EXPERIMENT,custom,Cost effectiveness evaluations based on cRCT d...,40,44,REFUTES,[CONCLUSIONS The published cost-effectiveness ...,train,cRCT
1730,embryonic stem cells (ESCs),TAXON,custom,Androgenetic haploid mouse embryonic stem cell...,27,54,SUPPORTS,[Our results demonstrate that AG-haESCs can be...,train,embryonic stem cells (ESCs)


In [13]:
"""    
    claim_id : int
    ner_text: str
    ner_label: str
    ner_model: str
    start_char: int
    end_char: int    
"""

for index, row in tqdm(df_scispacy_sentence_word_unq_ner_abr_filtered.iterrows()):
    # Access the 'sentence' and 'ner' columns of the row
    
    claim_txt = row['claim']
    ner_text = row['ner_text']
    ner_label = row['ner_label']
    ner_model = row['ner_model']
    start_char = row['start_char']
    end_char = row['end_char']
    term_in_claim = row['term_in_claim']
    # Do something with the data
    
    retrived_claim = Claim.get_claim_by_text(claims= unique_gold_claims, claim_text= claim_txt)
    if retrived_claim is None:
        print(f"Claim not found for claim_txt: {claim_txt}")
    else:
        entity = NEREntity(claim_id=retrived_claim.id, ner_text=ner_text, ner_label=ner_label, ner_model=ner_model, start_char=start_char, end_char=end_char)
        retrived_claim.ner_entities.append(entity)

101it [00:00, 2351.86it/s]

Claim not found for claim_txt: 1 in 5 million in UK have abnormal PrP positivity.
Claim not found for claim_txt: 1 in 5 million in UK have abnormal PrP positivity.
Claim not found for claim_txt: 1,000 genomes project enables mapping of genetic sequence variation consisting of rare variants with larger penetrance effects than common variants.
Claim not found for claim_txt: 1,000 genomes project enables mapping of genetic sequence variation consisting of rare variants with larger penetrance effects than common variants.
Claim not found for claim_txt: 1,000 genomes project enables mapping of genetic sequence variation consisting of rare variants with larger penetrance effects than common variants.
Claim not found for claim_txt: 1/2000 in UK have abnormal PrP positivity.
Claim not found for claim_txt: 1/2000 in UK have abnormal PrP positivity.
Claim not found for claim_txt: 32% of liver transplantation programs required patients to discontinue methadone treatment in 2001.
Claim not found f

AttributeError: 'NoneType' object has no attribute 'append'

In [13]:
for cur_claim in tqdm(unique_gold_claims[:10]):
    print(cur_claim.claim)

100%|██████████| 10/10 [00:00<00:00, 15839.52it/s]

10-20% of people with severe mental disorder receive no treatment in low and middle income countries.
25% of patients with melanoma and an objective response to PD-1 blockade will experience a progression in their melanoma.
50% of patients exposed to radiation have activated markers of myofibroblasts.
8% of burn patients are admitted for hospitalization and further treatment after appearing at hospital emergency wards or outpatient clinics.
A breast cancer patient's capacity to metabolize tamoxifen has no effect on treatment outcome.
A country's Vaccine Alliance (GAVI) eligibility is associated with accelerated adoption of the Hub vaccine.
A deficiency of folate decreases blood levels of homocysteine.
A deficiency of vitamin B6 increases blood levels of homocysteine.
APOE4 expression in iPSC-derived neurons results in increased tau phosphorylation.
AZT given with ribavirin alleviates anemia.
